#  <font color="red"> 1. Importing Libraries and Loading Data </font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# Load the Data
df = pd.read_csv("/kaggle/input/german-credit-data-with-risk/german_credit_data.csv",index_col=0)


#  <font color="red"> 2. Exploratory Data Analaysis </font>

In [2]:
df.sample(5)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
203,21,male,2,rent,little,little,902,12,education,bad
152,24,male,2,own,little,rich,5848,36,radio/TV,good
108,30,female,2,own,NaN,little,7721,24,furniture/equipment,good
488,35,male,1,rent,moderate,NaN,1418,10,car,good
638,34,female,2,own,little,NaN,1493,12,radio/TV,good


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Job               1000 non-null   int64 
 3   Housing           1000 non-null   object
 4   Saving accounts   817 non-null    object
 5   Checking account  606 non-null    object
 6   Credit amount     1000 non-null   int64 
 7   Duration          1000 non-null   int64 
 8   Purpose           1000 non-null   object
 9   Risk              1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 85.9+ KB


In [4]:
df.describe()

,Age,Job,Credit amount,Duration
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,35.546000,1.904000,3271.258000,20.903000
std,11.375469,0.653614,2822.736876,12.058814
min,19.000000,0.000000,250.000000,4.000000
25%,27.000000,2.000000,1365.500000,12.000000
50%,33.000000,2.000000,2319.500000,18.000000
75%,42.000000,2.000000,3972.250000,24.000000
max,75.000000,3.000000,18424.000000,72.000000


## <font color="blue"> 2.1 some plots </font>

### <font color="green"> Plot 1: Credit Risk Distribution   </font>

This plot displays the distribution of the target variable (credit_risk). It shows the count of each category (risk or no risk) in the dataset.

In [5]:
fig = px.histogram(df, x='Risk', color='Risk', title='Credit Risk Distribution')
fig.update_layout(xaxis_title='Credit Risk', yaxis_title='Count')
fig.show()

### <font color="green"> Plot 2: Correlation Heatmap  </font>

This plot shows a heatmap of the correlation between numeric features in the dataset. It helps to identify any significant relationships or patterns between the variables.

In [6]:
fig = px.imshow(df.corr(), labels=dict(color='Correlation'))
fig.update_layout(title='Correlation Heatmap')
fig.show()

### <font color="green"> Plot 3: Credit Amount by Credit Risk (Box Plot) </font>


This interactive plot displays the distribution of credit amounts based on the credit risk category. It allows you to compare the distribution of credit amounts between risk and no-risk categories.

In [7]:
fig = px.box(df, x='Risk', y='Credit amount', color='Risk', title='Credit Amount by Credit Risk')
fig.show()


### <font color="green"> Plot 4: Scatter Matrix   </font>

This interactive scatter matrix plot shows pairwise scatter plots for numeric features in the dataset. Each point is colored based on the credit risk category, enabling you to identify potential relationships and patterns between different variables.

In [8]:
fig = px.scatter_matrix(df, dimensions=['Duration', 'Credit amount', 'Job', 'Age'], color='Risk', title='Scatter Matrix')
fig.show()


###   <font color="green"> Plot 5:Categorical Distribution by Risk    </font> 
This interactive plot displays histograms for categorical columns, showing the frequency of each category and its relation to credit risk. Color indicates risk level. It aids in understanding how categorical variables relate to credit risk in the dataset.

In [9]:
categorical_columns = ['Sex', 'Housing', 'Saving accounts', 'Checking account', 'Purpose']

def visualize_categorical_columns(df, col_list, hue='Risk'):
    for col in col_list:
        fig = px.histogram(df, x=col, color=hue, title=f'{col} Distribution by Credit Risk')
        fig.show()

visualize_categorical_columns(df, categorical_columns)


###  <font color="green"> Plot 6: Credit Amount Distribution (Histogram) </font>

This interactive histogram displays the distribution of credit amounts in the dataset. It allows you to explore the range and frequency of different credit amount ranges, with the ability to view additional statistical information.

In [10]:
fig = px.histogram(df, x='Credit amount', nbins=20, color='Risk', marginal='box', title='Credit Amount Distribution')
fig.show()


## <font color="blue"> 2.2 Find number of missing values in each column </font>

In [11]:
# Find the number of null values for each column
num_null = df.isnull().sum()

# Find the percentage of null values for each column
percent_null = round((df.isnull().sum() / len(df)) * 100, 2)

# Combine the results into a single DataFrame
null_values = pd.concat([num_null, percent_null], axis=1, keys=['Number of Null Values', 'Percentage of Null Values'])

# Print the results
print(null_values)

                  Number of Null Values  Percentage of Null Values
Age                                   0                        0.0
Sex                                   0                        0.0
Job                                   0                        0.0
Housing                               0                        0.0
Saving accounts                     183                       18.3
Checking account                    394                       39.4
Credit amount                         0                        0.0
Duration                              0                        0.0
Purpose                               0                        0.0
Risk                                  0                        0.0


# <font color="red"> 3. Data Preprocessing </font>

## <font color="blue"> 3.1 Handling missing values </font>

In [12]:
# We will drop the column with 40% missing values because it has alot null values
# although we will be loosing some information. 
#Hopefully this does not causethe model to underfit in the future.
df.drop(['Checking account'], axis=1, inplace=True)

# The second column we will replace nan with word "non_exc"
df['Saving accounts'] = df['Saving accounts'].fillna('non_exc')


## <font color="blue"> 3.2 Encode categorical variables </font>

In [13]:
from sklearn.preprocessing import LabelEncoder

# Assuming your dataset is stored in a pandas DataFrame called 'df'
columns_to_encode = ['Sex', 'Housing',  'Purpose', 'Risk', 'Saving accounts']

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each column
for column in columns_to_encode:
    df[column] = label_encoder.fit_transform(df[column])

# The columns will now contain the encoded values



In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Age              1000 non-null   int64
 1   Sex              1000 non-null   int64
 2   Job              1000 non-null   int64
 3   Housing          1000 non-null   int64
 4   Saving accounts  1000 non-null   int64
 5   Credit amount    1000 non-null   int64
 6   Duration         1000 non-null   int64
 7   Purpose          1000 non-null   int64
 8   Risk             1000 non-null   int64
dtypes: int64(9)
memory usage: 78.1 KB


## <font color="blue"> 3.3 Feature scaling </font> 

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['Duration', 'Credit amount', 'Job', 'Age']] = scaler.fit_transform(df[['Duration', 'Credit amount', 'Job', 'Age']])

##  <font color="blue"> 3.4 Split the data into training and validation sets </font> 

In [16]:
X = df.drop("Risk", axis=1)
y = df["Risk"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# <font color="red"> 4. Modelling </font> 

## <font color="blue"> 4.1 Training Classification Models </font> 

In [17]:
# Logistic Regression
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Classification Trees
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)

# Neural Networks
nn_model = MLPClassifier()
nn_model.fit(X_train, y_train)


MLPClassifier()

## <font color="blue"> 4.2 Evaluate Models </font> 

In [18]:
# Choose one model from each technique and evaluate their performance
# Logistic Regression
logistic_pred = logistic_model.predict(X_val)
logistic_cm = confusion_matrix(y_val, logistic_pred)

# Classification Trees
tree_pred = tree_model.predict(X_val)
tree_cm = confusion_matrix(y_val, tree_pred)

# Neural Networks
nn_pred = nn_model.predict(X_val)
nn_cm = confusion_matrix(y_val, nn_pred)


## <font color="blue"> 4.3 Print Evaluation Results </font>

In [19]:
# Logistic Regression
print("Logistic Regression:")
print("Confusion Matrix:")
print(logistic_cm)
accuracy = (logistic_cm[0,0] + logistic_cm[1,1]) / np.sum(logistic_cm)
print("Accuracy:", accuracy)

# Classification Trees
print("\nClassification Trees:")
print("Confusion Matrix:")
print(tree_cm)
accuracy = (tree_cm[0,0] + tree_cm[1,1]) / np.sum(tree_cm)
print("Accuracy:", accuracy)

# Neural Networks
print("\nNeural Networks:")
print("Confusion Matrix:")
print(nn_cm)
accuracy = (nn_cm[0,0] + nn_cm[1,1]) / np.sum(nn_cm)
print("Accuracy:", accuracy)


Logistic Regression:
Confusion Matrix:
[[  9  50]
 [  5 136]]
Accuracy: 0.725

Classification Trees:
Confusion Matrix:
[[ 18  41]
 [ 41 100]]
Accuracy: 0.59

Neural Networks:
Confusion Matrix:
[[ 18  41]
 [ 11 130]]
Accuracy: 0.74


## <font color="blue"> 4.4 Determine the Model with the Highest Net Profit </font>

In [20]:
def calculate_net_profit_cm(confusion_matrix, true_labels):
    # Define the costs and benefits
    cost_true_negative = 0  # Cost of correctly predicting a low-risk applicant
    benefit_true_positive = 100  # Benefit of correctly predicting a high-risk applicant
    cost_false_negative = 50  # Cost of incorrectly predicting a high-risk applicant as low-risk
    benefit_false_positive = 0  # Benefit of incorrectly predicting a low-risk applicant as high-risk

    tn, fp, fn, tp = confusion_matrix.ravel()

    # Calculate the net profit
    net_profit = (
        tn * cost_true_negative +
        fp * benefit_false_positive +
        fn * cost_false_negative +
        tp * benefit_true_positive
    )

    # Calculate the net profit per record
    net_profit_per_record = net_profit / len(true_labels)

    return net_profit_per_record


validation_df = X_val.copy()
validation_df['Risk'] = y_val
true_labels = validation_df['Risk'].values

models = ["Logistic Regression", "Classification Trees", "Neural Networks"]
confusion_matrices = [logistic_cm, tree_cm, nn_cm]
best_model_index = np.argmax([calculate_net_profit_cm(cm, true_labels) for cm in confusion_matrices])
best_model = models[best_model_index]
print("Best Model:", best_model)

Best Model: Logistic Regression


# <font color="red"> 5. Improving Performance with Net Profit-based Credit Risk Selection</font>  
This code focuses on improving performance in credit risk classification by using estimated probabilities (propensities) from logistic regression to select credit risks. It calculates the net profit for each record in the validation set, creates a decile-wise lift chart incorporating net profit, and determines the maximum net profit and probability cutoff.



## <font color="blue"> 5.1 calculate_net_profit_proba </font>  
This cell contains the implementation of the calculate_net_profit_proba function, which assigns profit values based on the true labels and predicted probabilities. It uses conditional statements and np.where to assign different profit values for different outcomes.



In [21]:
def calculate_net_profit_proba(y_true, probabilities):
    # Assigning profit values for different outcomes
    true_positive_profit = 100  # Profit for correctly predicting a positive (Good) credit risk
    false_positive_profit = -50  # Loss for incorrectly predicting a positive (Good) credit risk
    true_negative_profit = 0  # No profit/loss for correctly predicting a negative (Bad) credit risk
    false_negative_profit = -100  # Loss for incorrectly predicting a negative (Bad) credit risk

    net_profit = np.where(y_true == 1,
                          np.where(probabilities >= 0.5, true_positive_profit, false_negative_profit),
                          np.where(probabilities >= 0.5, false_positive_profit, true_negative_profit))

    return net_profit


## <font color="blue"> 5.2 create_lift_chart </font> 
This cell contains the implementation of the create_lift_chart function, which creates a decile-wise lift chart based on the true labels and net profit. It sorts the data by net profit, calculates lift values at each decile, and returns the deciles and lift values.



In [22]:
def create_lift_chart(y_true, net_profit):
    # Create a decile-wise lift chart
    df = pd.DataFrame({'y_true': y_true, 'net_profit': net_profit})
    df_sorted = df.sort_values(by='net_profit', ascending=False)
    deciles = np.arange(10, 110, 10)
    lift_values = []
    total_profit = df_sorted['net_profit'].sum()

    for decile in deciles:
        index = int((decile / 100) * len(df_sorted))
        lift = df_sorted.iloc[:index]['net_profit'].sum() / total_profit
        lift_values.append(lift)

    return deciles, lift_values


## <font color="blue"> 5.3 determine_cutoff </font> 
This cell contains the implementation of the determine_cutoff function, which calculates the profit and probability cutoff based on the sorted probabilities and net profit. It uses np.argsort to sort the data, determines the cutoff index based on the specified percentile, and returns the cutoff profit and cutoff probability.



In [23]:
def determine_cutoff(probabilities, net_profit, percentile):
    sorted_indices = np.argsort(probabilities)[::-1]
    sorted_profit = net_profit[sorted_indices]
    cutoff_index = int(percentile * len(sorted_indices) / 100)
    cutoff_profit = sorted_profit[cutoff_index]
    cutoff_probability = np.percentile(probabilities, percentile)
    return cutoff_profit, cutoff_probability


## <font color="blue"> 5.4 Net Profit Calculation and Lift Chart Creation </font>
This cell calculates the net profit using calculate_net_profit_proba and logistic probabilities, creates a decile-wise lift chart using create_lift_chart, and determines the maximum net profit and probability cutoff.

In [24]:
logistic_probabilities = logistic_model.predict_proba(X_val)[:, 1]
net_profit = calculate_net_profit_proba(y_val, logistic_probabilities)
deciles, lift_values = create_lift_chart(y_val, net_profit)
max_net_profit_index = np.argmax(net_profit)
max_net_profit_decile = int((max_net_profit_index / len(net_profit)) * 10) + 1
cutoff_profit, cutoff_probability = determine_cutoff(logistic_probabilities, net_profit, percentile=80)


## <font color="blue"> 5.5 Print Results </font>
This cell prints the results, including the maximum net profit decile, cutoff profit at the 80th percentile, and cutoff probability at the 80th percentile.



In [25]:
print("Maximum Net Profit Decile:", max_net_profit_decile)
print("Cutoff Profit at 80th Percentile:", cutoff_profit)
print("Cutoff Probability at 80th Percentile:", cutoff_probability)


Maximum Net Profit Decile: 1
Cutoff Profit at 80th Percentile: -50
Cutoff Probability at 80th Percentile: 0.8206377577314894


## <font color="blue"> 5.6 Net Profit Lift Chart </font> 
This interactive plot illustrates the decile-wise lift in net profit achieved by using estimated probabilities from logistic regression for credit risk selection. It highlights the maximum net profit decile and the cutoff profit at the 80th percentile of probability.

In [26]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=deciles, y=lift_values, mode='lines+markers', name='Lift'))

fig.add_annotation(x=80, y=lift_values[7], text=f"80th Percentile (Cutoff)\nProfit: {cutoff_profit}, Probability: {cutoff_probability}",
                   showarrow=True, arrowhead=1, ax=0, ay=-50)
fig.update_layout(title='Lift Chart with Net Profit',
                  xaxis_title='Deciles',
                  yaxis_title='Lift',
                  xaxis=dict(showgrid=True, zeroline=True, title_font=dict(size=12)),
                  yaxis=dict(showgrid=True, zeroline=True, title_font=dict(size=12)),
                  hovermode='closest',
                  plot_bgcolor='rgba(0,0,0,0)',
                  legend=dict(x=0.1, y=0.9, bgcolor='rgba(255, 255, 255, 0.7)', bordercolor='black'))
fig.show()

##  <font color="blue"> 5.7 Result explain </font>

The results obtained from the previous code are as follows:

*  Maximum Net Profit Decile: 1 

This indicates that the record with the highest net profit belongs to the first decile. The decile-wise lift chart is divided into 10 equal parts (deciles), and the first decile represents the highest net profit among all the records in the validation set.


*  Cutoff Profit at 80th Percentile: -50 

This represents the net profit value at the 80th percentile of the sorted net profit values. It indicates the threshold net profit that can be used to categorize records as either positive (Good) or negative (Bad) credit risks. In this case, the cutoff profit at the 80th percentile is -50, meaning that records with a net profit of -50 or higher can be considered as potentially positive credit risks.


*  Cutoff Probability at 80th Percentile: 0.8206377577314894 

This represents the probability value at the 80th percentile of the sorted logistic probabilities. It indicates the threshold probability that can be used to categorize records as either positive (Good) or negative (Bad) credit risks. In this case, the cutoff probability at the 80th percentile is 0.8206, meaning that records with a probability of 0.8206 or higher can be considered as potentially positive credit risks.

<div align="center">
    <h1><font color="#e74c3c">***</font> Notebook Conclusion <font color="#e74c3c">***</font></h1>
</div>
<font color="#2ecc71"><b>Conclusion:</b></font>

Congratulations on reaching the end of this Kaggle notebook! Throughout this analysis, we explored the dataset, performed data preprocessing, built models, and evaluated their performance. We gained valuable insights and made data-driven decisions to solve the problem at hand.


<font color="#2ecc71"><b>Thank You!</b></font>

We would like to express our gratitude to the Kaggle community . Your support, feedback, and collaboration have been invaluable. If you have any questions or suggestions, please feel free to reach out. Happy analyzing!

<font color="#2ecc71"><b>Stay Curious, Keep Learning!</b></font>

